<CENTER><img src="../../images/ATLASOD.gif" style="width:50%"></CENTER>

# How to rediscover the Higgs boson yourself!
This notebook uses ATLAS Open Data http://opendata.atlas.cern to show you the steps to rediscover the Higgs boson yourself!

The datasets used in this notebook have already been filtered to include at least 2 photons per event, so that processing is quicker.

This analysis loosely follows the discovery of the Higgs boson by ATLAS https://arxiv.org/pdf/1207.7214.pdf (mostly Section 5 and 5.1)

Feynman diagram pictures are borrowed from our friends at https://www.particlezoo.net

<CENTER><img src="Hyy_feynman.png" style="width:40%"></CENTER>

## First time setup on your computer (no need on mybinder)
This first cell only needs to be run the first time you open this notebook on your computer. 

If you close Jupyter and re-open on the same computer, you won't need to run this first cell again.

If you open on binder, you don't need to run this cell.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --user pip # update the pip package installer
!{sys.executable} -m pip install -U numpy pandas uproot matplotlib lmfit --user # install required packages

## To setup everytime
Cell -> Run All Below

to be done every time you re-open this notebook

In [ ]:
import uproot # for reading .root files
import pandas as pd # to store data as dataframe
import time # to measure time to analyse
import math # for mathematical functions such as square root
import numpy as np # # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from lmfit.models import PolynomialModel, GaussianModel # for the signal and background fits
from matplotlib.ticker import MaxNLocator,AutoMinorLocator # for minor ticks

General definitions of luminosity, fraction of data used, where to access the input files

In [ ]:
lumi = 0.5 # fb-1 # data_A only
#lumi = 1.9 # fb-1 # data_B only
#lumi = 2.9 # fb-1 # data_C only
#lumi = 4.7 # fb-1 # data_D only
#lumi = 10 # fb-1 # data_A,data_B,data_C,data_D

fraction = 0.8 # reduce this is you want the code to run quicker

#tuple_path = "Input/GamGam/Data/" # local 
tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/" # web address

samples to process

In [ ]:
samples = {

    'data': {
        'list' : ['data_A'] #'data_B','data_C','data_D' # add if you want more data
    },

}

Define function to get data from files

In [ ]:
def get_data_from_files():

    data = {} # define empty dictionary to hold all data
    for s in samples: # loop over samples defined above
        print('Processing '+s+' samples') # print which sample is being processed
        frames = [] # define empty list to hold data
        for val in samples[s]['list']: # loop over each file
            fileString = tuple_path+val+".GamGam.root" # file name to open
            if fileString != "": # if this file name has worked
                temp = read_file(fileString,val) # call the function read_file defined below
                frames.append(temp) # append dataframe returned from read_file to list of dataframes
            else: # fileString == ""
                print("Error: "+val+" not found!") # print error message
        data[s] = pd.concat(frames) # concatenate list of dataframes together into one dataframe
    
    return data # return dictionary of dataframes

Define function to calculate diphoton invariant mass

In [ ]:
def calc_myy(photon_pt,photon_eta,photon_phi,photon_E):
    # first photon is [0], 2nd photon is [1] etc
    px_0 = photon_pt[0]*math.cos(photon_phi[0]) # x-component of photon[0] momentum
    py_0 = photon_pt[0]*math.sin(photon_phi[0]) # y-component of photon[0] momentum
    pz_0 = photon_pt[0]*math.sinh(photon_eta[0]) # z-component of photon[0] momentum
    px_1 = photon_pt[1]*math.cos(photon_phi[1]) # x-component of photon[1] momentum
    py_1 = photon_pt[1]*math.sin(photon_phi[1]) # y-component of photon[1] momentum
    pz_1 = photon_pt[1]*math.sinh(photon_eta[1]) # z-component of photon[1] momentum
    sumpx = px_0 + px_1 # x-component of diphoton momentum
    sumpy = py_0 + py_1 # y-component of diphoton momentum
    sumpz = pz_0 + pz_1 # z-component of diphoton momentum 
    sump = math.sqrt(sumpx**2 + sumpy**2 + sumpz**2) # magnitude of diphoton momentum 
    sumE = photon_E[0] + photon_E[1] # energy of diphoton system
    return math.sqrt(sumE**2 - sump**2)/1000 #/1000 to go from MeV to GeV

## Changing an already uncommented cut

If you change a cut: Cell -> Run All Below

If you uncomment a cut here, you also need to uncomment the corresponding cut in the cell above.

In [ ]:
# Cut on number of photons
# paper: "The data used in this channel are selected using a diphoton trigger, which requires two clusters"
def cut_photon_n (photon_n):
# want to throw away events where photon_n does not equal 2
# exclamation mark (!) means "not"
# so != means "not equal to"
    return photon_n != 2

# Cut on pseudorapidity outside the fiducial region
# paper: "Photon candidates are reconstructed in the fiducial region |η| < 2.37"
def cut_photon_eta_fiducial(photon_eta):
# want to throw away events where modulus of photon_eta > 2.37
    return photon_eta[0] > 2.37 or photon_eta[1] > 2.37 or photon_eta[0] < -2.37 or photon_eta[1] < -2.37

# Cut on pseudorapidity in barrel/end-cap transition region
# paper: "excluding the calorimeter barrel/end-cap transition region 1.37 < |η| < 1.52"
def cut_photon_eta_transition(photon_eta):
# want to throw away events where modulus of photon_eta between 1.37 and 1.52
    if photon_eta[0] < 1.52 and photon_eta[0] > 1.37: return True # True means throw away
    elif photon_eta[1] < 1.52 and photon_eta[1] > 1.37: return True
    elif photon_eta[0] > -1.52 and photon_eta[0] < -1.37: return True
    elif photon_eta[1] < -1.37 and photon_eta[1] > -1.52: return True
    else: return False
    
# Cut on Transverse momentum
# paper: "The leading (sub-leading) photon candidate is required to have ET > 40 GeV (30 GeV)"
def cut_photon_pt(photon_pt):
# want to throw away events where photon_pt[0] < 40000 MeV or photon_pt[1] < 30000 MeV
# first photon is [0], 2nd photon is [1] etc
    return photon_pt[0] < 40000 or photon_pt[1] < 30000

# Cut on photon reconstruction quality
# paper: "Photon candidates are required to pass identification criteria"
def cut_photon_reconstruction(photon_isTightID):
# isTightID == True means a photon that has been identified as being well reconstructed
# want to throw away events where it is false for one or both photons
    return photon_isTightID[0] == False or photon_isTightID[1] == False

# Cut on energy isolation
# paper: "Photon candidates are required to have an isolation transverse energy of less than 4 GeV"
def cut_isolation_et(photon_etcone20):
# want to throw away events where isolation eT > 4000 MeV
    return photon_etcone20[0] > 4000 or photon_etcone20[1] > 4000
    
# Cut on reconstructed invariant mass lower limit
# paper: "in the diphoton invariant mass range between 100 GeV and 160 GeV"
def cut_mass_lower(myy):
# want to throw away minimum invariant reconstructed mass < 100 GeV
    return myy < 100

# Cut on reconstructed invariant mass upper limit
# paper: "in the diphoton invariant mass range between 100 GeV and 160 GeV"
def cut_mass_upper(myy):
# want to throw away maximum invariant reconstructed mass > 160 GeV
    return myy > 160

## Uncommenting a new cut 

If you add a cut: Cell -> Run All Below

In [ ]:
def read_file(path,sample):
    start = time.time() # start the clock
    print("\tProcessing: "+sample) # print which sample is being processed
    data_all = pd.DataFrame() # define empty pandas DataFrame to hold all data for this sample
    mc = uproot.open(path)["mini"] # open the tree called mini
    numevents = uproot.numentries(path, "mini") # number of events
    for data in mc.iterate(["photon_n","photon_trigMatched","photon_pt","photon_eta","photon_phi","photon_E",
                            "photon_isTightID","photon_etcone20"], # add more variables here if you want to use them
                           flatten=False, # make JaggedArrays lists
                           entrysteps=2500000, # number of events in a batch to process
                           outputtype=pd.DataFrame, # choose output type as pandas DataFrame
                           entrystop=numevents*fraction): # process up to numevents*fraction

        nIn = len(data.index) # number of events in this batch
        print('\t initial number of events:\t\t\t\t',nIn)
        
        # Cut on number of photons using the function cut_photon_n defined above
        fail = data[ np.vectorize(cut_photon_n)(data.photon_n)].index
        data.drop(fail, inplace=True)
        print('\t after requiring exactly 2 photons:\t\t\t',len(data.index))
        
        # Cut on photon reconstruction quality using the function cut_photon_reconstruction defined above
        fail = data[ np.vectorize(cut_photon_reconstruction)(data.photon_isTightID)].index
        data.drop(fail, inplace=True)
        print('\t after requiring tight photons:\t\t\t\t',len(data.index))
        
        # Cut on transverse momentum of the photons using the function cut_photon_pt defined above
        fail = data[ np.vectorize(cut_photon_pt)(data.photon_pt)].index
        data.drop(fail, inplace=True)
        print('\t after photon pt requirement:\t\t\t\t',len(data.index))
        
        # Cut on energy isolation using the function cut_isolation_et defined above
        fail = data[ np.vectorize(cut_isolation_et)(data.photon_etcone20)].index
        data.drop(fail, inplace=True)
        print('\t after photon isolation requirement:\t\t\t',len(data.index))
        
        # Cut on pseudorapidity outside fiducial region using the function cut_photon_eta_fiducial defined above
        fail = data[ np.vectorize(cut_photon_eta_fiducial)(data.photon_eta)].index
        data.drop(fail, inplace=True)
        print('\t after requiring photons within fiducial region:\t',len(data.index))
        
        # Cut on pseudorapidity inside barrel/end-cap transition region using the function cut_photon_eta_transition
        fail = data[ np.vectorize(cut_photon_eta_transition)(data.photon_eta)].index
        data.drop(fail, inplace=True)
        print('\t after throwing away photons in transition region:\t',len(data.index))
        
        # Calculate reconstructed diphoton invariant mass using the function calc_myy defined above
        data['myy'] = np.vectorize(calc_myy)(data.photon_pt,data.photon_eta,data.photon_phi,data.photon_E)

        # Cut on lower limit of reconstructed invariant mass using the function cut_mass_lower
        fail = data[ np.vectorize(cut_mass_lower)(data.myy)].index
        data.drop(fail, inplace=True)
        print('\t after cut on diphoton mass lower limit:\t\t',len(data.index))
        # Cut on upper limit of reconsructed invariant mass using the function cut_mass_upper
        fail = data[ np.vectorize(cut_mass_upper)(data.myy)].index
        data.drop(fail, inplace=True)
        print('\t after cut on diphoton mass upper limit:\t\t',len(data.index))

        # dataframe contents can be printed at any stage like this
        #print(data)

        # dataframe column can be printed at any stage like this
        #print(data['photon_pt'])

        # multiple dataframe columns can be printed at any stage like this
        #print(data[['photon_pt','photon_eta']])

        nOut = len(data.index) # number of events passing cuts in this batch
        data_all = data_all.append(data) # append dataframe from this batch to the dataframe for the whole sample
        elapsed = time.time() - start # time taken to process
        print("\t\t nIn: "+str(nIn)+",\t nOut: \t"+str(nOut)+"\t in "+str(round(elapsed,1))+"s") # events before and after
    
    return data_all # return dataframe containing events passing all cuts

This is where the processing happens

In [ ]:
start = time.time() # time at start of whole processing
data = get_data_from_files() # process all files
elapsed = time.time() - start # time after whole processing
print("Time taken: "+str(round(elapsed,1))+"s") # print total time taken to process every file

## Make a change to plotting
If you only want a make a change in the plot: Cell -> Run All Below

In [ ]:
myy = { # dictionary containing plotting parameters for the myy histogram
    # change plotting parameters                                                                                                     
    'bin_width':2, # width of each histogram bin
    'num_bins':30, # number of histogram bins
    'xrange_min':100, # minimum on the x-axis
    'xlabel':r'$\mathrm{m_{\gamma\gamma}}$ [GeV]', # x-axis label

    # change aesthetic parameters if you want                                                                                        
    'y_label_x_position':-0.09, # 0.09 to the left of y axis                                                                         
    'legend_loc':'lower left',                      
    'linear_top_margin':1.1 # to decrease the separation between data and the top of the figure, pick a number closer to 1           
}

hist_dict = {'myy': myy} # add a histogram here if you want it plotted

Define function to plot the data

In [ ]:
def plot_data(data):
    
    plot_label = r'$H \rightarrow \gamma\gamma$' # label to write on the plot
    
    # *******************
    # general definitions (shouldn't need to change)
    lumi_used = str(lumi*fraction) # luminosity to write on the plot   

    for x_variable,hist in hist_dict.items(): # access the dictionary of histograms defined in the cell above

        h_bin_width = hist['bin_width'] # get the bin width defined in the cell above
        h_num_bins = hist['num_bins'] # get the number of bins defined in the cell above
        h_xrange_min = hist['xrange_min'] # get the x-range minimum defined in the cell above
        h_xlabel = hist['xlabel'] # get the x-axis label defined in the cell above
        h_y_label_x_position = hist['y_label_x_position'] # get the x-position of the y-axis label defined in the cell above
        h_legend_loc = hist['legend_loc'] # get the legend location defined in the cell above
        h_linear_top_margin = hist['linear_top_margin'] # to decrease the separation between data and the top of the figure, pick a number closer to 1

        bins = [ h_xrange_min + x*h_bin_width for x in range(h_num_bins+1) ] # bin limits
        bin_centres = [ h_xrange_min+h_bin_width/2 + x*h_bin_width for x in range(h_num_bins) ] # bin centres

        data_x,_ = np.histogram( data['data'][x_variable].values, bins=bins ) # histogram the data
        data_x_errors = np.sqrt( data_x ) # statistical error on the data
    
        # data fit
        polynomial_mod = PolynomialModel( 4 ) # 4th order polynomial
        gaussian_mod = GaussianModel() # Gaussian
        bin_centres_array = np.asarray(bin_centres) # array of bin centres
        # set initial guesses for the parameters of the polynomial model
        pars = polynomial_mod.guess(data_x, # data to use to guess parameter values
                                    x=bin_centres_array, c0=data_x.max(), c1=0, c2=0, c3=0, c4=0 )
        # set initial guesses for the parameters of the Gaussian model
        pars += gaussian_mod.guess(data_x, # data to use to guess parameter values
                                   x=bin_centres_array, amplitude=91.7, center=125., sigma=2.4 )
        model = polynomial_mod + gaussian_mod # combined model
        out = model.fit(data_x, # data to be fit
                        pars, # guesses for the parameters
                        x=bin_centres_array, weights=1/data_x_errors ) # fit the model to the data
    
        # background part of fit
        params_dict = out.params.valuesdict() # get the parameters from the fit to data
        c0 = params_dict['c0'] # c0 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
        c1 = params_dict['c1'] # c1 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
        c2 = params_dict['c2'] # c2 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
        c3 = params_dict['c3'] # c3 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
        c4 = params_dict['c4'] # c4 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
        # get the background only part of the fit to data
        background = c0 + c1*bin_centres_array + c2*bin_centres_array**2 + c3*bin_centres_array**3 + c4*bin_centres_array**4

        signal_x = data_x - background # data fit - background fit = signal fit
    
    
        # *************
        # Main plot 
        # *************
        plt.axes([0.1,0.3,0.85,0.65]) # left, bottom, width, height 
        main_axes = plt.gca() # get current axes
        # plot the data points
        main_axes.errorbar( x=bin_centres, y=data_x, yerr=data_x_errors, fmt='ko', label='Data' ) 
        # plot the signal + background fit
        main_axes.plot(bin_centres, # x
                       out.best_fit, # y
                       '-r', # single red line
                       label='Sig+Bkg Fit ($m_H=125$ GeV)' )
        # plot the background only fit
        main_axes.plot(bin_centres, # x
                       background, # y
                       '--r', # dashed red line
                       label='Bkg (4th order polynomial)' )
        
        main_axes.set_xlim( left=h_xrange_min, right=bins[-1] ) # set the x-limit of the main axes
        main_axes.xaxis.set_minor_locator( AutoMinorLocator() ) # separation of x axis minor ticks
        # set the axis tick parameters for the main axes
        main_axes.tick_params(which='both', # ticks on both x and y axes
                              direction='in', # Put ticks inside and outside the axes
                              top=True, # draw ticks on the top axis
                              labeltop=False, # don't draw tick labels on top axis
                              labelbottom=False, # don't draw tick labels on bottom axis
                              right=True, # draw ticks on right axis
                              labelright=False ) # don't draw tick labels on right axis
        if len( h_xlabel.split('[') ) > 1: # if x-axis has units
            y_units = ' '+h_xlabel[h_xlabel.find('[')+1:h_xlabel.find(']')]
        else: y_units = '' # if x-axis is unitless
        main_axes.set_ylabel('Events / '+str(h_bin_width)+y_units, fontname='sans-serif',
                             horizontalalignment='right', y=1.0, fontsize=11 ) # write y-axis label for main axes
        main_axes.set_ylim( bottom=0, top=(np.amax(data_x)+math.sqrt(np.amax(data_x)))*h_linear_top_margin ) # set the y axis limit for the main axes
        main_axes.yaxis.set_minor_locator( AutoMinorLocator() ) # set minor ticks on the y axis of the main axes
        main_axes.yaxis.get_major_ticks()[0].set_visible(False) # avoid displaying y=0 on the main axes
        
        # Add text 'ATLAS Open Data' on plot
        plt.text(0.2, # x
                 0.97, # y
                 'ATLAS Open Data', # text
                 transform=main_axes.transAxes, # coordinate system used is that of main_axes
                 horizontalalignment='left', verticalalignment='top', family='sans-serif', fontsize=13 ) 
        # Add text 'for education' on plot
        plt.text(0.2, # x
                 0.9, # y
                 'for education', # text
                 transform=main_axes.transAxes, # coordinate system used is that of main_axes
                 horizontalalignment='left', verticalalignment='top', family='sans-serif', style='italic',
                 fontsize=8 ) 
        # Add energy and luminosity
        plt.text(0.2, # x
                 0.86, # y
                 '$\sqrt{s}=13\,\mathrm{TeV},\;\int L\,dt=$'+lumi_used+'$\,\mathrm{fb}^{-1}$', # text
                 transform=main_axes.transAxes, # coordinate system used is that of main_axes
                 horizontalalignment='left', verticalalignment='top', family='sans-serif' ) 
        # Add a label for the analysis carried out
        plt.text(0.2, # x
                 0.78, # y
                 plot_label, # text 
                 transform=main_axes.transAxes, # coordinate system used is that of main_axes
                 horizontalalignment='left', verticalalignment='top', family='sans-serif' ) 
    
        # draw the legend
        main_axes.legend(frameon=False, # no box around the legend
                         loc=h_legend_loc ) # legend location 
    
    
        # *************
        # Data-Bkg plot 
        # *************
        plt.axes([0.1,0.1,0.85,0.2]) # left, bottom, width, height
        sub_axes = plt.gca() # get the current axes
        sub_axes.yaxis.set_major_locator( MaxNLocator(nbins='auto', symmetric=True) ) # set the y axis to be symmetric about Data-Background=0
        sub_axes.errorbar( x=bin_centres, y=signal_x, yerr=data_x_errors, fmt='ko' ) # plot Data-Background
        # draw the fit to data
        sub_axes.plot(bin_centres, # x
                      out.best_fit-background, # y
                      '-r' ) # single red line
        # draw the background only fit
        sub_axes.plot(bin_centres, # x
                      background-background, # y
                      '--r' )  # dashed red line
        sub_axes.set_xlim( left=h_xrange_min, right=bins[-1] ) # set the x-axis limits on the sub axes
        sub_axes.xaxis.set_minor_locator( AutoMinorLocator() ) # separation of x-axis minor ticks
        sub_axes.xaxis.set_label_coords( 0.9,-0.2 ) # (x,y) of x axis label # 0.2 down from x-axis
        sub_axes.set_xlabel( h_xlabel, fontname='sans-serif', fontsize=11 ) # x-axis label
        # set the tick parameters for the sub axes
        sub_axes.tick_params(which='both', # ticks on both x and y axes
                             direction='in', # Put ticks inside and outside the axes
                             top=True, # draw ticks on the top axis
                             labeltop=False, # don't draw tick labels on top axis
                             right=True, # draw ticks on right axis
                             labelright=False ) # don't draw tick labels on right axis 
        sub_axes.yaxis.set_minor_locator( AutoMinorLocator() ) # separation of y-axis minor ticks
        sub_axes.set_ylabel( 'Events-Bkg', fontname='sans-serif', x=1, fontsize=11 ) # y-axis label on the sub axes
        
        
        # Generic features for both plots
        main_axes.yaxis.set_label_coords( h_y_label_x_position, 1 ) # x,y coordinates of the y-axis label on the main axes
        sub_axes.yaxis.set_label_coords( h_y_label_x_position, 0.5 ) # x,y coordinates of the y-axis label on the sub axes

        plt.savefig( 'Hyy_'+x_variable+'.pdf', bbox_inches='tight' ) # save the plot
    
        print( 'chi^2 = ' + str(out.chisqr) ) # get the chi squared of the fit
        print( 'gaussian centre = ' + str(params_dict['center']) ) # centre of the fitted Gaussian
        print( 'gaussian sigma = ' + str(params_dict['sigma']) ) # sigma of the fitted Gaussian
        print( 'gaussian fwhm = ' + str(params_dict['fwhm']) ) # Full-Width-at-Half-Maximum of the fitted Gaussian
    
    return

Call the function to plot the data

In [ ]:
plot_data(data)